# Imports

In [1]:
#webscraping
import requests
from bs4 import BeautifulSoup

import pandas as pd


url = 'https://www.federalreserve.gov/releases/h41/current/h41.htm'
get = requests.get(url)
bs = BeautifulSoup(get.content, 'html.parser')
print(get.ok)

True


# Data

## Values

In [2]:
#<tr> with <td> have <th> that are the names for the data
#data in <td> are in <p>
#first table with tr p has date
data = bs.select('.H41Release > tr > td > p')

In [288]:
count = 0
for d in data:
    if '1,826,212' in d.text:
        break;
    count += 1

In [263]:
def findstartlocation(string, vals=data):
    count = 0
    for d in vals:
        if string in d.text:
            break;
        count += 1
    return count

In [289]:
count

332

In [293]:
data[337]

<p>Guaranteed by Fannie Mae, Freddie Mac, and Ginnie Mae. The current face value shown is the remaining principal balance of the securities.</p>

In [265]:
#133, 188
#204 7 * 12
count

204

In [20]:
table2vals = data[133:189]
table3vals = data[204:236]
table4vals = data[240:324]
table5vals = data[332:337]
#table6vals = data

In [260]:
count

188

## Row Names

In [ ]:
#td id names
#t2r1
#t2r2
#t2r3
#...
#t2r34
#t3r1
#
#headers tXc0


In [5]:
tablenum = 2
table1_headers = bs.findAll('th', {'headers' : 't{}c0'.format(tablenum)})

In [13]:
desiredtables = [2, 3, 5, 8, 10, 12, 13, 15, 16, 18]
tableHeaders = []
for tablenum in desiredtables:
    tableHeaders.append(bs.findAll('th', {'headers' : 't{}c0'.format(tablenum)}))

## create dict for data

## table 1

In [3]:
type(data)

list

In [ ]:
df_table3 = createTable(table3vals, tableHeaders[2])


In [81]:
tableHeaders[2][0].select('p')[0].text

'Securities held in custody for foreign official and international accounts'

['3,391,236', '+\xa0\xa0\xa0\xa02,828', '-\xa0\xa0\xa072,101', '3,384,052']

In [100]:
def createTable(data, headers, columns=columnNames):
    table = []
    table1 = {}
    c = 0
    for x in headers:
        try:
            vals = []
            for num, col in enumerate(columns):
                vals.append(data[c * len(columns) + num ].text.strip())
            table1[x.select('p')[0].text] = vals
            c += 1
        except:
            #can't use enumerate since ('p')[0] may be out of index
            continue
    df = pd.DataFrame(table1).T
    df.columns = columns
    return df

In [7]:
table = []
table1 = {}
count = 0
for x in table1_headers:
    try: #index out of range for no p elements in stuffs
        table1[x.select('p')[0].text] = [data[count * 4 + 1].text.strip(),
                                        data[count * 4 + 2].text.strip(),
                                        data[count * 4 + 3].text.strip(),
                                        data[count * 4 + 4].text.strip()]
        count += 1
    except:
        #can't use enumerate since ('p')[0] may fail
        continue

In [8]:
df_table1 = pd.DataFrame(table1).T

In [9]:
columnNames = ['Week Ended May 27, 2020',
              'Change from May 20, 2020',
              'Change from May 29, 2020',
              'Wednesday May 27, 2020']

In [10]:
df_table1.columns = columnNames

## table 2

In [83]:
table2 = {}
rowNames2 = []
count = 0
for x in tableHeaders[1]:
    try:
        table2[x.select('p')[0].text] = [table2vals[count * 4].text.strip(),
                                            table2vals[count * 4 + 1].text.strip(),
                                            table2vals[count * 4 + 2].text.strip(),
                                            table2vals[count * 4 + 3].text.strip()]
        count += 1
    except:
            continue
        

In [19]:
count = 0
table2[x.select('p')[0].text] = [table2vals[count * 4 + 1].text.strip(),
                                    table2vals[count * 4 + 2].text.strip(),
                                    table2vals[count * 4 + 3].text.strip(),
                                    table2vals[count * 4 + 4].text.strip()]

NameError: name 'table2vals' is not defined

In [84]:
df_table2 = pd.DataFrame(table2).T

In [85]:
df_table2 = pd.DataFrame(table2).T
df_table2.columns = columnNames

In [86]:
df_table2.head()

,"Week Ended May 27, 2020","Change from May 20, 2020","Change from May 29, 2020","Wednesday May 27, 2020"
Currency in circulation11,"1,944,218","+ 10,951","+ 206,641","1,948,196"
Reverse repurchase agreements12,"245,506","- 15,342","- 21,894","243,976"
Foreign official and international accounts,"244,887","- 13,964","- 17,870","239,650"
Others,619,"- 1,378","- 4,024","4,326"
Treasury cash holdings,239,- 34,- 31,207


## table 3


In [90]:
tableHeaders[2]

[<th headers="t5c0" id="t5r1" style="font-family:Arial;font-size:8pt;vertical-align:bottom;border-top:solid 1px black;border-right:solid 1px black;padding-left:1.8245em;text-indent:-1.8245em;"><p>Securities held in custody for foreign official and international accounts</p></th>,
 <th headers="t5c0 t5r1" id="t5r2" style="font-family:Arial;font-size:8pt;vertical-align:bottom;border-right:solid 1px black;padding-left:0.7184em;"><p>Marketable U.S. Treasury securities<sup>1</sup></p></th>,
 <th headers="t5c0 t5r1" id="t5r3" style="font-family:Arial;font-size:8pt;vertical-align:bottom;border-right:solid 1px black;padding-left:0.7184em;"><p>Federal agency debt and mortgage-backed securities<sup>2</sup></p></th>,
 <th headers="t5c0 t5r1" id="t5r4" style="font-family:Arial;font-size:8pt;vertical-align:bottom;border-right:solid 1px black;padding-left:0.7184em;"><p>Other securities<sup>3</sup></p></th>,
 <th headers="t5c0" id="t5r5" style="font-family:Arial;font-size:8pt;vertical-align:bottom;bo

In [95]:
df_table3 = createTable(table3vals, tableHeaders[2])

In [97]:
df_table3.tail()

,"Week Ended May 27, 2020","Change from May 20, 2020","Change from May 29, 2020","Wednesday May 27, 2020"
Other securities3,"85,420",+ 475,"+ 9,581","85,447"
Securities lent to dealers,"32,694",+ 347,"+ 11,237","32,934"
Overnight facility4,"32,694",+ 347,"+ 11,237","32,934"
U.S. Treasury securities,"32,694",+ 347,"+ 11,237","32,934"
Federal agency debt securities,0,0,0,0


In [131]:
assliabNames = ['Within 15 days',
               '16 to 90 days',
               '91 days to 1 year',
               'Over 1 year to 5 years',
               'Over 5 years to 10 years',
               'Over 10 years',
               'All']
df_table4 = createTable(table4vals,tableHeaders[3], columns = assliabNames)

6
13
20
27
34
41
48
55
62
69
76
83
fail at 12
fail at 13
fail at 14
fail at 15


In [115]:
7 * 12

84

In [114]:
len(table4vals), len(assliabNames)

(84, 7)

In [123]:
allvals = []
for x in range(int(len(table4vals) / len(assliabNames))):
    vals = []
    for num, col in enumerate(assliabNames):
        vals.append(table4vals[x * len(assliabNames) + num ].text.strip())
    allvals.append(vals)
        

In [130]:
def createTable(data, headers, columns=columnNames):
    table = []
    table1 = {}
    for c, x in enumerate(headers):
        try:
            vals = []
            for num, col in enumerate(columns):
                vals.append(data[c * len(columns) + num ].text.strip())
            table1[x.select('p')[0].text] = vals
            print(c * len(columns) + num)
        except:
            print('fail at {}'.format(c))
            #can't use enumerate since ('p')[0] may be out of index
            continue
    df = pd.DataFrame(table1).T
    df.columns = columns
    return df

In [124]:
allvals

[['9,777', '36,411', '11,497', '49,211', '0', '...', '106,896'],
 ['55,608',
  '279,039',
  '564,220',
  '1,577,552',
  '719,915',
  '913,177',
  '4,109,512'],
 ['-\xa0\xa0\xa011,251',
  '+\xa0\xa0\xa0\xa04,864',
  '+\xa0\xa0\xa0\xa06,377',
  '+\xa0\xa0\xa0\xa09,486',
  '+\xa0\xa0\xa0\xa08,657',
  '+\xa0\xa0\xa0\xa02,047',
  '+\xa0\xa0\xa020,181'],
 ['0', '0', '0', '0', '1,436', '911', '2,347'],
 ['0', '0', '0', '0', '0', '0', '0'],
 ['0', '0', '4', '2,217', '80,749', '1,752,140', '1,835,110'],
 ['0',
  '0',
  '-\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa01',
  '-\xa0\xa0\xa0\xa0\xa0\xa0\xa074',
  '-\xa0\xa0\xa0\xa01,156',
  '-\xa0\xa0\xa026,499',
  '-\xa0\xa0\xa027,731'],
 ['0', '4,255', '0', '...', '...', '...', '4,255'],
 ['139,551', '41,550', '...', '...', '...', '...', '181,101'],
 ['126,741', '322,205', '0', '0', '0', '0', '448,946'],
 ['243,976', '0', '...', '...', '...', '...', '243,976'],
 ['0', '0', '0', '...', '...', '...', '0']]

In [107]:
table4vals

[<p>     9,777</p>,
 <p>    36,411</p>,
 <p>    11,497</p>,
 <p>    49,211</p>,
 <p>         0</p>,
 <p>...</p>,
 <p>   106,896</p>,
 <p>    55,608</p>,
 <p>   279,039</p>,
 <p>   564,220</p>,
 <p> 1,577,552</p>,
 <p>   719,915</p>,
 <p>   913,177</p>,
 <p> 4,109,512</p>,
 <p>-   11,251</p>,
 <p>+    4,864</p>,
 <p>+    6,377</p>,
 <p>+    9,486</p>,
 <p>+    8,657</p>,
 <p>+    2,047</p>,
 <p>+   20,181</p>,
 <p>         0</p>,
 <p>         0</p>,
 <p>         0</p>,
 <p>         0</p>,
 <p>     1,436</p>,
 <p>       911</p>,
 <p>     2,347</p>,
 <p>         0</p>,
 <p>         0</p>,
 <p>         0</p>,
 <p>         0</p>,
 <p>         0</p>,
 <p>         0</p>,
 <p>         0</p>,
 <p>         0</p>,
 <p>         0</p>,
 <p>         4</p>,
 <p>     2,217</p>,
 <p>    80,749</p>,
 <p> 1,752,140</p>,
 <p> 1,835,110</p>,
 <p>         0</p>,
 <p>         0</p>,
 <p>-        1</p>,
 <p>-       74</p>,
 <p>-    1,156</p>,
 <p>-   26,499</p>,
 <p>-   27,731</p>,
 <p>         0</p>,
 <p>   

In [111]:
table4vals[51]

<p>         0</p>

In [127]:
df_table4

,Within 15 days,16 to 90 days,91 days to 1 year,Over 1 year to 5 years,Over 5 years to 10 years,Over 10 years,All
Loans1,"9,777","36,411","11,497","49,211",0,...,"106,896"
U.S. Treasury securities2,"55,608","279,039","564,220","1,577,552","719,915","913,177","4,109,512"
Holdings,"139,551","41,550",...,...,...,...,"181,101"
Weekly changes,"126,741","322,205",0,0,0,0,"448,946"
Federal agency debt securities3,0,0,0,0,0,0,0
Mortgage-backed securities4,0,"4,255",0,...,...,...,"4,255"
Commercial paper held by Commercial,"243,976",0,...,...,...,...,"243,976"
Paper Funding Facility II LLC5,0,0,0,...,...,...,0


In [74]:
tableHeaders[2]

[<th headers="t5c0" id="t5r1" style="font-family:Arial;font-size:8pt;vertical-align:bottom;border-top:solid 1px black;border-right:solid 1px black;padding-left:1.8245em;text-indent:-1.8245em;"><p>Securities held in custody for foreign official and international accounts</p></th>,
 <th headers="t5c0 t5r1" id="t5r2" style="font-family:Arial;font-size:8pt;vertical-align:bottom;border-right:solid 1px black;padding-left:0.7184em;"><p>Marketable U.S. Treasury securities<sup>1</sup></p></th>,
 <th headers="t5c0 t5r1" id="t5r3" style="font-family:Arial;font-size:8pt;vertical-align:bottom;border-right:solid 1px black;padding-left:0.7184em;"><p>Federal agency debt and mortgage-backed securities<sup>2</sup></p></th>,
 <th headers="t5c0 t5r1" id="t5r4" style="font-family:Arial;font-size:8pt;vertical-align:bottom;border-right:solid 1px black;padding-left:0.7184em;"><p>Other securities<sup>3</sup></p></th>,
 <th headers="t5c0" id="t5r5" style="font-family:Arial;font-size:8pt;vertical-align:bottom;bo

In [103]:
table4vals

[<p>     9,777</p>,
 <p>    36,411</p>,
 <p>    11,497</p>,
 <p>    49,211</p>,
 <p>         0</p>,
 <p>...</p>,
 <p>   106,896</p>,
 <p>    55,608</p>,
 <p>   279,039</p>,
 <p>   564,220</p>,
 <p> 1,577,552</p>,
 <p>   719,915</p>,
 <p>   913,177</p>,
 <p> 4,109,512</p>,
 <p>-   11,251</p>,
 <p>+    4,864</p>,
 <p>+    6,377</p>,
 <p>+    9,486</p>,
 <p>+    8,657</p>,
 <p>+    2,047</p>,
 <p>+   20,181</p>,
 <p>         0</p>,
 <p>         0</p>,
 <p>         0</p>,
 <p>         0</p>,
 <p>     1,436</p>,
 <p>       911</p>,
 <p>     2,347</p>,
 <p>         0</p>,
 <p>         0</p>,
 <p>         0</p>,
 <p>         0</p>,
 <p>         0</p>,
 <p>         0</p>,
 <p>         0</p>,
 <p>         0</p>,
 <p>         0</p>,
 <p>         4</p>,
 <p>     2,217</p>,
 <p>    80,749</p>,
 <p> 1,752,140</p>,
 <p> 1,835,110</p>,
 <p>         0</p>,
 <p>         0</p>,
 <p>-        1</p>,
 <p>-       74</p>,
 <p>-    1,156</p>,
 <p>-   26,499</p>,
 <p>-   27,731</p>,
 <p>         0</p>,
 <p>   

## cleaning

In [179]:
table1

{'Reserve Bank credit': ['7,059,561',
  '+\xa0\xa0137,525',
  '+3,239,597',
  '7,058,713'],
 'Securities held outright1': ['5,958,725',
  '+\xa0\xa0\xa086,155',
  '+2,279,834',
  '5,946,969'],
 'U.S. Treasury securities': ['4,101,108',
  '+\xa0\xa0\xa027,033',
  '+1,986,418',
  '4,109,512'],
 'Bills2': ['326,044', '0', '+\xa0\xa0325,994', '326,044'],
 'Notes and bonds, nominal2': ['3,481,681',
  '+\xa0\xa0\xa024,558',
  '+1,505,116',
  '3,489,174'],
 'Notes and bonds, inflation-indexed2': ['256,594',
  '+\xa0\xa0\xa0\xa02,400',
  '+\xa0\xa0141,233',
  '257,451'],
 'Inflation compensation3': ['36,789',
  '+\xa0\xa0\xa0\xa0\xa0\xa0\xa075',
  '+\xa0\xa0\xa014,075',
  '36,843'],
 'Federal agency debt securities2': ['2,347', '0', '0', '2,347'],
 'Mortgage-backed securities4': ['1,855,270',
  '+\xa0\xa0\xa059,122',
  '+\xa0\xa0293,416',
  '1,835,110'],
 'Unamortized premiums on securities held outright5': ['304,609',
  '+\xa0\xa0\xa0\xa04,442',
  '+\xa0\xa0171,805',
  '304,256'],
 'Unamortiz

In [61]:
#remove unic issues '\xa0'
def removeUnicode(x):
    if '\xa0' in str(x):
        return x.replace('\xa0', '')
    else:
        return x
def removePlus(x):
    if '+' in str(x):
        return x.replace('+', '')
    else:
        return x
def removeComma(x):
    if ',' in str(x):
        return x.replace(',', '')
    else:
        return x


In [87]:
#apply methods and change to int
all_dfs = [df_table1, df_table2]
for df in all_dfs:
    for c in columnNames:
        df[c] = df[c].apply(removeUnicode)
        df[c] = df[c].apply(removePlus)
        df[c] = df[c].apply(removeComma)
        df[c] = df[c].astype(int)

In [67]:
df_table1.head()

,"Week Ended May 27, 2020","Change from May 20, 2020","Change from May 29, 2020","Wednesday May 27, 2020"
Reserve Bank credit,7059561,137525,3239597,7058713
Securities held outright1,5958725,86155,2279834,5946969
U.S. Treasury securities,4101108,27033,1986418,4109512
Bills2,326044,0,325994,326044
"Notes and bonds, nominal2",3481681,24558,1505116,3489174


In [88]:
df_table2.head()

,"Week Ended May 27, 2020","Change from May 20, 2020","Change from May 29, 2020","Wednesday May 27, 2020"
Currency in circulation11,1944218,10951,206641,1948196
Reverse repurchase agreements12,245506,-15342,-21894,243976
Foreign official and international accounts,244887,-13964,-17870,239650
Others,619,-1378,-4024,4326
Treasury cash holdings,239,-34,-31,207
